In [105]:
import pandas as pd


In [106]:
df = pd.read_csv('jobOffers.csv', index_col=0)

In [107]:
df.head()


,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote
0,Java Software Developer,Intempt Technologies,23.2k - 34.9k CZK,NaN,"Backend, java",Junior,Work language:angličtina,NaN,Na dálku
1,C++ Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku
2,C# Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku
3,Java developer u Medvědů,Coding Bear,60k - 120k CZK,NaN,"Backend, java","Mid, Senior",Work language:česky,"Praha, Vítkova 241/10 • Praha, Vítkova 241/10",NaN
4,Senior Backend Developer,DX Heroes,60k - 100k CZK,NaN,"Backend, node","Mid, Senior","Work language:česky, angličtina","Praha, Rohanské nábř. 19 • Remote• Praha, Roh...",Na dálku


In [108]:
#salary
#dropping the non-paid offers
df = df.drop(df[df['salaryRange'] == 'neplacená stáž'].index)

#getting lower and upper ranges of the possible salary
df['salLower'] = df['salaryRange'].str.split(" - ").str[0].str.split("  ").str[0]
df['salUpper'] = df['salaryRange'].str.split(" - ").str[1].str.split("  ").str[0]

df['salLower'] = df['salLower'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)
df['salUpper'].fillna(df['salLower'], inplace = True)
df['salUpper'] = df['salUpper'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

#getting the average of the salary range
df['salAvg'] = (df['salUpper'] + df['salLower'])/2


In [109]:
#category and main programming language
df['category'] = df['category+progLang'].str.split(", ").str[0]
df['mainProgLang'] = df['category+progLang'].str.split(", ").str[1]
df.head()

,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote,salLower,salUpper,salAvg,category,mainProgLang
0,Java Software Developer,Intempt Technologies,23.2k - 34.9k CZK,NaN,"Backend, java",Junior,Work language:angličtina,NaN,Na dálku,23200,34900,29050.0,Backend,java
1,C++ Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku,70000,140000,105000.0,Backend,C#
2,C# Developer,Sourcein,70k - 140k CZK,NaN,"Backend, C#","Mid, Senior",Work language:česky,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Covid-time na dálku,70000,140000,105000.0,Backend,C#
3,Java developer u Medvědů,Coding Bear,60k - 120k CZK,NaN,"Backend, java","Mid, Senior",Work language:česky,"Praha, Vítkova 241/10 • Praha, Vítkova 241/10",NaN,60000,120000,90000.0,Backend,java
4,Senior Backend Developer,DX Heroes,60k - 100k CZK,NaN,"Backend, node","Mid, Senior","Work language:česky, angličtina","Praha, Rohanské nábř. 19 • Remote• Praha, Roh...",Na dálku,60000,100000,80000.0,Backend,node


In [110]:
# adjusting the names of the seniority column
df["seniorityAdj"] = df["seniority"].str.strip().replace({'Trainee, Junior': 'Junior', 'Trainee': 'Junior','Senior, Expert':'Senior',
                                                          'Expert':'Senior', 'Mid, Senior':'Mid Advanced'})
df.groupby(['seniorityAdj']).size()     

seniorityAdj
Junior           69
Mid             190
Mid Advanced    243
Senior          214
dtype: int64

In [111]:
#Work language
df['language'] = df['workLanguage'].str.split(':').str[1]
df['language'] = df['language'].str.split(',')

df.groupby(['workLanguage']).size()
langList = ['angličtina','česky']
for lang in langList:
    df[lang] = df['workLanguage'].str.contains(lang)

#['angličtina', 'česky' ]


165

In [112]:
#converting less demanded programming languages to 'other'
mask = df['mainProgLang'].map(df['mainProgLang'].value_counts())<20
df['mainProgLangAdj']=df['mainProgLang'].mask(mask,'other')
counts = df.groupby(['mainProgLangAdj']).size().sort_values(ascending=False)
print(counts)

mainProgLangAdj
other         239
java          121
python         83
javascript     74
.net           59
react          57
php            48
dtype: int64


In [120]:
#df['location'].str.contains('Praha|Prague')
df['remoteBool'] = df['remote'].notnull().astype('boolean')

In [121]:
#adjusting bonus variable, few obsevations have included the bonus in absolute term e.g. 2000CZK instead in the percentages
mask = df['Bonus'].str.contains('%', na = False)==False
df['BonusAdj']=df['Bonus'].mask(mask,'No bonus').str.strip()
df['BonusAdj'].unique()


array(['No bonus', '+15%', '+10%', '+50%', '+5%', '+25%', '+1%', '+12%',
       '+20%', '+13%', '+7%', '+30%', '+2%'], dtype=object)

In [122]:
cleanDF = df[['jobTitles', 'company', 'BonusAdj', 'salLower', 'salUpper','salAvg', 'category', 'mainProgLangAdj', 'seniorityAdj', 'angličtina', 'česky', 'remoteBool']]

In [123]:
# drop all job offers, that only require some foreign language outside Czech or English
cleanDF = cleanDF.loc[cleanDF.angličtina | cleanDF.česky ]


In [124]:
cleanDF.head()

,jobTitles,company,BonusAdj,salLower,salUpper,salAvg,category,mainProgLangAdj,seniorityAdj,angličtina,česky,remoteBool
0,Java Software Developer,Intempt Technologies,No bonus,23200,34900,29050.0,Backend,java,Junior,True,False,True
1,C++ Developer,Sourcein,No bonus,70000,140000,105000.0,Backend,other,Mid Advanced,False,True,True
2,C# Developer,Sourcein,No bonus,70000,140000,105000.0,Backend,other,Mid Advanced,False,True,True
3,Java developer u Medvědů,Coding Bear,No bonus,60000,120000,90000.0,Backend,java,Mid Advanced,False,True,False
4,Senior Backend Developer,DX Heroes,No bonus,60000,100000,80000.0,Backend,other,Mid Advanced,True,True,True


In [126]:
cleanDF.to_csv('jobOffersClean.csv')